# 🧩 Advanced ToT Demos: Sudoku & Maze Solving

Supplementary lab for **Day 5** – showcasing Tree‑/Graph‑of‑Thought reasoning on two classic search problems:

1. **Sudoku** – constraint‑propagation + ToT branching  
2. **Maze Solving** – path‑finding visualized as a Graph‑of‑Thought

Colab‑ready; pure‑Python (no heavy deps) plus optional LLM hooks.

## 🔧 0. Setup

Install lightweight deps for visualization.

In [ ]:
%pip -q install --upgrade networkx matplotlib openai
import os, random, itertools, math, copy, time
import networkx as nx
import matplotlib.pyplot as plt
openai_key = os.getenv("OPENAI_API_KEY")


---

## 1️⃣ Sudoku via Tree‑of‑Thoughts

We use a simple **choose‑cell, branch on candidate digits** strategy.

* **State**: 9×9 grid (list of lists, 0 = blank)  
* **Expand**: pick blank with fewest legal digits, branch on each candidate  
* **Score**: +1 per filled cell (optionally ask LLM to judge likelihood of success)  
* **Goal**: no blanks & valid grid


In [ ]:
def print_grid(g):
    for r in g: print(' '.join(str(c or '.') for c in r))
    print()

def possible(grid, r, c, d):
    # row/col/box constraints
    if any(grid[r][i]==d for i in range(9)): return False
    if any(grid[i][c]==d for i in range(9)): return False
    br,bc = 3*(r//3), 3*(c//3)
    for i in range(br, br+3):
        for j in range(bc, bc+3):
            if grid[i][j]==d: return False
    return True

def expand_sudoku(grid):
    # find most constrained blank
    blanks=[(r,c) for r in range(9) for c in range(9) if grid[r][c]==0]
    if not blanks: return []
    r,c = min(blanks, key=lambda rc: sum(possible(grid,rc[0],rc[1],d) for d in range(1,10)))
    children=[]
    for d in range(1,10):
        if possible(grid,r,c,d):
            new=copy.deepcopy(grid)
            new[r][c]=d
            children.append(new)
    return children

def score_grid(grid): 
    filled = sum(1 for r in grid for c in r if c!=0)
    return filled + random.random()*0.01  # tie‑break

def is_goal(grid): 
    return all(grid[r][c]!=0 for r in range(9) for c in range(9))

def tot_sudoku(grid, beam=5, depth=81):
    frontier=[(grid, score_grid(grid))]
    for d in range(depth):
        new=[]
        for state,_ in frontier:
            if is_goal(state): return state
            for child in expand_sudoku(state):
                new.append((child, score_grid(child)))
        frontier = sorted(new, key=lambda x:-x[1])[:beam]
    return None

puzzle = [
    [5,3,0, 0,7,0, 0,0,0],
    [6,0,0, 1,9,5, 0,0,0],
    [0,9,8, 0,0,0, 0,6,0],

    [8,0,0, 0,6,0, 0,0,3],
    [4,0,0, 8,0,3, 0,0,1],
    [7,0,0, 0,2,0, 0,0,6],

    [0,6,0, 0,0,0, 2,8,0],
    [0,0,0, 4,1,9, 0,0,5],
    [0,0,0, 0,8,0, 0,7,9],
]

print("Puzzle:")
print_grid(puzzle)
solution = tot_sudoku(puzzle, beam=20)
print("Solution:")
print_grid(solution)

#### 📝 Experiments

* Increase/decrease `beam` width – how does speed vs. success trade‑off look?  
* Switch `score_grid` to call an **LLM**: ask it to rate likelihood that this partial grid leads to a valid solution.

---

## 2️⃣ Maze Solving as Graph‑of‑Thought

We model the maze as a grid. Each cell is a node; edges connect walkable neighbors.

We’ll:

1. Build the graph  
2. Run plain BFS (**linear** reasoning)  
3. Visualize the entire search graph (GoT) and highlight the found path


In [ ]:
maze_str = """###########
#S#     #E#
# # ### # #
# #   #   #
# ### ##### 
#         #
###########"""  # S=start, E=end, #=wall, ' ' = open

maze = [list(row) for row in maze_str.splitlines()]
H,W = len(maze), len(maze[0])

start=end=None
for i in range(H):
    for j in range(W):
        if maze[i][j]=='S': start=(i,j)
        if maze[i][j]=='E': end=(i,j)

dirs=[(-1,0),(1,0),(0,-1),(0,1)]

G=nx.Graph()
for r in range(H):
    for c in range(W):
        if maze[r][c] != '#':
            G.add_node((r,c))
            for dr,dc in dirs:
                nr,nc=r+dr,c+dc
                if 0<=nr<H and 0<=nc<W and maze[nr][nc]!='#':
                    G.add_edge((r,c),(nr,nc))

# BFS
from collections import deque
parent={}
dq=deque([start]); parent[start]=None
while dq:
    cur=dq.popleft()
    if cur==end: break
    for nb in G.neighbors(cur):
        if nb not in parent:
            parent[nb]=cur
            dq.append(nb)

# reconstruct path
path=[]
node=end
while node:
    path.append(node); node=parent[node]
path=path[::-1]

plt.figure(figsize=(6,6))
pos={ (r,c):(c,-r) for r,c in G.nodes}
nx.draw(G,pos,node_size=20, node_color='lightgrey', width=0.5)
nx.draw_networkx_nodes(G,pos,nodelist=path,node_color='red',node_size=60)
plt.title("Maze Graph‑of‑Thought – red = solution path")
plt.axis('off')
plt.show()

#### 📝 Challenge

* Replace `maze_str` with your own ASCII maze.  
* Write a **ToT variant**: branch on 2‑3 promising moves at each depth (heuristic = Manhattan distance to goal), compare with BFS.  
* Visualize the pruned GoT and discuss explored vs. skipped nodes.

---

## 🔗 More to Explore

* Norvig, “Solving Every Sudoku Puzzle” – constraint propagation + search  
* DFS/BFS vs. A* heuristics – when does ToT give gains?  
* Yao et al., “Tree‑of‑Thought…”, 2023 – includes Sudoku & game tasks  
* Long & Bosch, “Graph‑of‑Thought…”, 2024 – memory‑aware agent reasoning
